In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import roc_auc_score, f1_score
from scipy.stats import entropy
from gensim.models import Word2Vec
import time
import gc
import os

import tqdm                                                                                                   
import concurrent.futures
import multiprocessing

pd.set_option('display.max_columns', None)
num_processes = multiprocessing.cpu_count()
print("total cpu count", +num_processes) 

os.environ['NUMEXPR_MAX_THREADS'] = '10'

from core.utils import timeit, reduce_mem

total cpu count 8


In [2]:
path = "/media/ryan/F/deep-learning-data/turing/vedio-predict/"

path_sub = path + 'sub/'
path_npy = path + 'npy/'
path_data = path + 'raw/'
path_model = path + 'model/'
path_result = path + 'result/'
path_pickle = path + 'pickle/'
path_profile = path + 'profile/'

debug_small = False

if debug_small:
    train_df = pd.read_pickle(path_pickle + 'train_small.pickle')
    test_df = pd.read_pickle(path_pickle + 'test_small.pickle')
    # app = pd.read_pickle(path_pickle + 'app_small.pickle')
    # user = pd.read_pickle(path_pickle + 'user_small.pickle')
else:
    train_df = pd.read_pickle(path_pickle + 'train.pickle')
    test_df = pd.read_pickle(path_pickle + 'test.pickle')
    # app = pd.read_pickle(path_pickle + 'app.pickle')
    # user = pd.read_pickle(path_pickle + 'user.pickle')


In [3]:
train_df

,id,target,timestamp,deviceid,newsid,guid,pos,app_version,device_vendor,netmodel,osversion,lng,lat,device_version,ts
0,1,0,NaN,8b2d7f2aed47ab32e9c6ae4f5ae00147,8008333091915950969,9a2c909ebc47aec49d9c160cdb4a6572,1,2.1.5,HONOR,g4,9,1.125385e+02,3.783793e+01,STF-AL00,1573298086436
1,2,0,NaN,8b2d7f2aed47ab32e9c6ae4f5ae00147,8008333091915950969,9a2c909ebc47aec49d9c160cdb4a6572,1,2.1.5,HONOR,w,9,1.117312e+02,3.562274e+01,STF-AL00,1573298087570
2,3,0,NaN,832aaa33cdf4a0938ba2c795eb3ffefd,4941885624885390992,d51a157d2b1e0e9aed4dd7f9900b85b2,2,1.9.9,vivo,w,8.1.0,4.940656e-324,4.940656e-324,V1818T,1573377075934
3,4,0,NaN,832aaa33cdf4a0938ba2c795eb3ffefd,6088376349846612406,d51a157d2b1e0e9aed4dd7f9900b85b2,1,1.9.9,vivo,w,8.1.0,4.940656e-324,4.940656e-324,V1818T,1573377044359
4,5,0,NaN,67dd9dac18cce1a6d79e8f20eefd98ab,5343094189765291622,625dc45744f59ddbc3ec8df161217188,0,2.1.1,xiaomi,w,9,1.167509e+02,3.656831e+01,Redmi Note 7,1573380989662
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11376676,11376677,0,NaN,04d9da051fd19038824ff5e62231999b,2243097017724653319,22f0487299a136c32ca50881a9e7e837,0,2.1.5,HONOR,o,9,1.057023e+02,2.962280e+01,HRY-AL00a,1573208907130
11376677,11376678,0,NaN,04d9da051fd19038824ff5e62231999b,2263062300471437930,22f0487299a136c32ca50881a9e7e837,4,2.1.5,HONOR,o,9,1.057023e+02,2.962280e+01,HRY-AL00a,1573209232467
11376678,11376679,0,NaN,04d9da051fd19038824ff5e62231999b,3805019430149772895,22f0487299a136c32ca50881a9e7e837,0,2.1.5,HONOR,w,9,1.057023e+02,2.962280e+01,HRY-AL00a,1573253363853
11376679,11376680,0,NaN,04d9da051fd19038824ff5e62231999b,3834534693777959342,22f0487299a136c32ca50881a9e7e837,1,2.1.5,HONOR,o,9,1.057023e+02,2.962280e+01,HRY-AL00a,1573208896400


In [4]:
# train_df = train_df[train_df.deviceid.str[-1] == '1']
# test_df = test_df[test_df.deviceid.str[-1] == '1']

In [5]:
# train_df

In [8]:
print('=============================================== read train ===============================================')
t = time.time()
# train_df = pd.read_csv('dataset/train.csv')
train_df['date'] = pd.to_datetime(
    train_df['ts'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x / 1000)))
)
train_df['day'] = train_df['date'].dt.day

# 训练集中，day=7的个数为11个，day=8的为3,674,871。 day9，10也是解决40w
# day=7占比不到1/百万，属于异常情况，去掉合理？ 线上的表现又会如何，为啥不是直接删除，这样有点过了
# 这里为啥只是改了day，不去直接改ts和timestamp呢？
train_df.loc[train_df['day'] == 7, 'day'] = 8
train_df['hour'] = train_df['date'].dt.hour
train_df['minute'] = train_df['date'].dt.minute
train_num = train_df.shape[0]
labels = train_df['target'].values
print('runtime:', time.time() - t)

=============================================== read train ===============================================
runtime: 21.164100646972656


In [7]:
print('=============================================== click data ===============================================')
click_df = train_df[train_df['target'] == 1].sort_values('timestamp').reset_index(drop=True)
click_df['exposure_click_gap'] = click_df['timestamp'] - click_df['ts']
click_df = click_df[click_df['exposure_click_gap'] >= 0].reset_index(drop=True)
click_df['date'] = pd.to_datetime(
    click_df['timestamp'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x / 1000)))
)
click_df['day'] = click_df['date'].dt.day
# 同上对day==7的修改
click_df.loc[click_df['day'] == 7, 'day'] = 8

del train_df['target'], train_df['timestamp']

# 这里为啥要把click_df的这些字段删除呢？
for f in ['date', 'exposure_click_gap', 'timestamp', 'ts', 'target', 'hour', 'minute']:
    del click_df[f]
print('runtime:', time.time() - t)

=============================================== click data ===============================================
runtime: 27.9391667842865


In [8]:
print('=============================================== read test ===============================================')
test_df['date'] = pd.to_datetime(
    test_df['ts'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x / 1000)))
)
test_df['day'] = test_df['date'].dt.day

# 测试集中，day=10的个数为32个，day=11的为3,653,560占比 1/十万，属于异常情况，去掉合理
test_df.loc[test_df['day'] == 10, 'day'] = 11
test_df['hour'] = test_df['date'].dt.hour
test_df['minute'] = test_df['date'].dt.minute
df = pd.concat([train_df, test_df], axis=0, ignore_index=False)
del train_df, test_df, df['date']
gc.collect()
print('runtime:', time.time() - t)

=============================================== read test ===============================================
runtime: 38.73506569862366


In [9]:
print('============================================= category encoding =============================================')
df['lng_lat'] = df['lng'].astype('str') + '_' + df['lat'].astype('str')
del df['guid']
click_df['lng_lat'] = click_df['lng'].astype('str') + '_' + click_df['lat'].astype('str')
sort_df = df.sort_values('ts').reset_index(drop=True)
cate_cols = [
    'deviceid', 'newsid', 'pos', 'app_version', 'device_vendor',
    'netmodel', 'osversion', 'device_version', 'lng', 'lat', 'lng_lat'
]
for f in cate_cols:
    print(f)
    map_dict = dict(zip(df[f].unique(), range(df[f].nunique())))
    df[f] = df[f].map(map_dict).fillna(-1).astype('int32')
    click_df[f] = click_df[f].map(map_dict).fillna(-1).astype('int32')
    sort_df[f] = sort_df[f].map(map_dict).fillna(-1).astype('int32')
    df[f + '_count'] = df[f].map(df[f].value_counts())
df = reduce_mem(df)
click_df = reduce_mem(click_df)
sort_df = reduce_mem(sort_df)
print('runtime:', time.time() - t)


============================================= category encoding =============================================
deviceid
newsid
pos
app_version
device_vendor
netmodel
osversion
device_version
lng
lat
lng_lat
2580.12 Mb, 1390.40 Mb (46.11 %)
77.91 Mb, 46.97 Mb (39.71 %)
1204.05 Mb, 673.70 Mb (44.05 %)
runtime: 169.73485732078552


In [10]:
print('============================================= feat engineer =============================================')

print('*************************** history stats ***************************')
for f in [
    ['deviceid'],
    ['pos', 'deviceid'],
    # ...
]:
    print('------------------ {} ------------------'.format('_'.join(f)))

    # 对前一天的点击次数进行统计
    tmp = click_df[f + ['day', 'id']].groupby(f + ['day'], as_index=False)['id'].agg(
        {'_'.join(f) + '_prev_day_click_count': 'count'})
    tmp['day'] += 1
    df = df.merge(tmp, on=f + ['day'], how='left')
    df['_'.join(f) + '_prev_day_click_count'] = df['_'.join(f) + '_prev_day_click_count'].fillna(0)
    df.loc[df['day'] == 8, '_'.join(f) + '_prev_day_click_count'] = None

    # 对前一天的曝光量进行统计
    tmp = df[f + ['day', 'id']].groupby(f + ['day'], as_index=False)['id'].agg(
        {'_'.join(f) + '_prev_day_count': 'count'})
    tmp['day'] += 1
    df = df.merge(tmp, on=f + ['day'], how='left')
    df['_'.join(f) + '_prev_day_count'] = df['_'.join(f) + '_prev_day_count'].fillna(0)
    df.loc[df['day'] == 8, '_'.join(f) + '_prev_day_count'] = None

    # 计算前一天的点击率
    df['_'.join(f) + '_prev_day_ctr'] = df['_'.join(f) + '_prev_day_click_count'] / (
            df['_'.join(f) + '_prev_day_count'] + df['_'.join(f) + '_prev_day_count'].mean())

    del tmp
    print('runtime:', time.time() - t)
del click_df
df = reduce_mem(df)

============================================= feat engineer =============================================
*************************** history stats ***************************
------------------ deviceid ------------------
runtime: 181.16225504875183
------------------ pos_deviceid ------------------
runtime: 195.30335974693298
2078.43 Mb, 1562.40 Mb (24.83 %)


In [11]:
print('*************************** exposure_ts_gap ***************************')
for f in [
    ['deviceid'], ['newsid'], ['lng_lat'],
    ['pos', 'deviceid'], ['pos', 'newsid'], ['pos', 'lng_lat'],
    ['pos', 'deviceid', 'lng_lat'],
    ['netmodel', 'deviceid'],
    ['pos', 'netmodel', 'deviceid'],
    ['netmodel', 'lng_lat'], ['deviceid', 'lng_lat'],
    ['netmodel', 'deviceid', 'lng_lat'], ['pos', 'netmodel', 'lng_lat'],
    ['pos', 'netmodel', 'deviceid', 'lng_lat']
]:
    print('------------------ {} ------------------'.format('_'.join(f)))

    tmp = sort_df[f + ['ts']].groupby(f)
    # 前x次、后x次曝光到当前的时间差
    for gap in [1, 2, 3, 5, 10]:
        sort_df['{}_prev{}_exposure_ts_gap'.format('_'.join(f), gap)] = tmp['ts'].shift(0) - tmp['ts'].shift(gap)
        sort_df['{}_next{}_exposure_ts_gap'.format('_'.join(f), gap)] = tmp['ts'].shift(-gap) - tmp['ts'].shift(0)
        tmp2 = sort_df[
            f + ['ts', '{}_prev{}_exposure_ts_gap'.format('_'.join(f), gap),
                 '{}_next{}_exposure_ts_gap'.format('_'.join(f), gap)]
            ].drop_duplicates(f + ['ts']).reset_index(drop=True)
        df = df.merge(tmp2, on=f + ['ts'], how='left')
        del sort_df['{}_prev{}_exposure_ts_gap'.format('_'.join(f), gap)]
        del sort_df['{}_next{}_exposure_ts_gap'.format('_'.join(f), gap)]
        del tmp2

    del tmp
    df = reduce_mem(df)
    print('runtime:', time.time() - t)
del df['ts']
gc.collect()

*************************** exposure_ts_gap ***************************
------------------ deviceid ------------------
2709.12 Mb, 2135.76 Mb (21.16 %)
runtime: 314.36668968200684
------------------ newsid ------------------
3282.48 Mb, 2709.12 Mb (17.47 %)
runtime: 444.183531999588
------------------ lng_lat ------------------
3855.84 Mb, 3282.48 Mb (14.87 %)
runtime: 566.2674934864044
------------------ pos_deviceid ------------------
4429.20 Mb, 3855.84 Mb (12.94 %)
runtime: 698.9450771808624
------------------ pos_newsid ------------------
5002.56 Mb, 4429.20 Mb (11.46 %)
runtime: 853.4127724170685
------------------ pos_lng_lat ------------------
5575.92 Mb, 5002.56 Mb (10.28 %)
runtime: 999.0461049079895
------------------ pos_deviceid_lng_lat ------------------
6149.28 Mb, 5575.92 Mb (9.32 %)
runtime: 1154.560673713684
------------------ netmodel_deviceid ------------------
6722.64 Mb, 6149.28 Mb (8.53 %)
runtime: 1307.9333600997925
------------------ pos_netmodel_deviceid -----

0

In [12]:
df.to_pickle(path_pickle + "df_081_without_cross.pickle")
print("success build df_081_without_cross.pickle")

success build sample.pickle


In [3]:
## 重新加载数据处理
df = pd.read_pickle(path_pickle + 'df_081_without_cross.pickle')
cate_cols = [
    'deviceid', 'newsid', 'pos', 'app_version', 'device_vendor',
    'netmodel', 'osversion', 'device_version', 'lng', 'lat', 'lng_lat'
]

del df['id']
gc.collect()

0

In [12]:
# from pyspark.sql import Row
# from pyspark import SparkConf
# from pyspark import SparkContext

# conf = SparkConf()
# conf.setAppName("[陈亮时/149675]-[tf_format_test]")
# sc = SparkContext(conf=conf)
    
# l = [('Ankit',25),('Jalfaizy',22),('saurabh',20),('Bala',26)]
# rdd = sc.parallelize(l)
# people = rdd.map(lambda x: Row(name=x[0], age=int(x[1])))
# schemaPeople = sqlContext.createDataFrame(people)

In [13]:
# print('*************************** cross feat (second order) ***************************')
# # 二阶交叉特征，可以继续做更高阶的交叉。
# def build_cross_feat(df, f, col):
#     print('------------------ {} {} ------------------'.format(f, col))
#     df = df.merge(df[[f, col]].groupby(f, as_index=False)[col].agg({
#         'cross_{}_{}_nunique'.format(f, col): 'nunique',
#         'cross_{}_{}_ent'.format(f, col): lambda x: entropy(x.value_counts() / x.shape[0])  # 熵
#     }), on=f, how='left')
#     if 'cross_{}_{}_count'.format(f, col) not in df.columns.values and 'cross_{}_{}_count'.format(col,
#                                                                                                   f) not in df.columns.values:
#         df = df.merge(df[[f, col, 'id']].groupby([f, col], as_index=False)['id'].agg({
#             'cross_{}_{}_count'.format(f, col): 'count'  # 共现次数
#         }), on=[f, col], how='left')
#     if 'cross_{}_{}_count_ratio'.format(col, f) not in df.columns.values:
#         df['cross_{}_{}_count_ratio'.format(col, f)] = df['cross_{}_{}_count'.format(f, col)] / df[
#             f + '_count']  # 比例偏好
#     if 'cross_{}_{}_count_ratio'.format(f, col) not in df.columns.values:
#         df['cross_{}_{}_count_ratio'.format(f, col)] = df['cross_{}_{}_count'.format(f, col)] / df[
#             col + '_count']  # 比例偏好
#     df['cross_{}_{}_nunique_ratio_{}_count'.format(f, col, f)] = df['cross_{}_{}_nunique'.format(f, col)] / df[
#         f + '_count']
#     print('runtime:', time.time() - t)
#     df = reduce_mem(df)
#     return df
        
# cross_cols = ['deviceid', 'newsid', 'pos', 'netmodel', 'lng_lat']
# f_col_tuple_list = []
# for f in cross_cols:
#     for col in cross_cols:
#         if col == f:
#             continue
#         f_col_tuple_list.append((f, col))
        
# print(f_col_tuple_list)
# # with concurrent.futures.ProcessPoolExecutor(num_processes) as pool:
# #     df = list(tqdm.tqdm(pool.map(build_cross_feat, cross_cols, chunksize=10, total=df.shape[0])))
# for tuple_o in tqdm.tqdm(f_col_tuple_list):
#     print(tuple_o)
#     df = build_cross_feat(df, tuple_o[0], tuple_o[1])

# del df['id']
# gc.collect()

In [14]:
print('*************************** cross feat (second order) ***************************')
# 二阶交叉特征，可以继续做更高阶的交叉。
cross_cols = ['deviceid', 'newsid', 'pos', 'netmodel', 'lng_lat']
for f in cross_cols:
    for col in cross_cols:
        if col == f:
            continue
        print('------------------ {} {} ------------------'.format(f, col))
        df = df.merge(df[[f, col]].groupby(f, as_index=False)[col].agg({
            'cross_{}_{}_nunique'.format(f, col): 'nunique',
            'cross_{}_{}_ent'.format(f, col): lambda x: entropy(x.value_counts() / x.shape[0])  # 熵
        }), on=f, how='left')
        if 'cross_{}_{}_count'.format(f, col) not in df.columns.values and 'cross_{}_{}_count'.format(col,
                                                                                                      f) not in df.columns.values:
            df = df.merge(df[[f, col, 'id']].groupby([f, col], as_index=False)['id'].agg({
                'cross_{}_{}_count'.format(f, col): 'count'  # 共现次数
            }), on=[f, col], how='left')
        if 'cross_{}_{}_count_ratio'.format(col, f) not in df.columns.values:
            df['cross_{}_{}_count_ratio'.format(col, f)] = df['cross_{}_{}_count'.format(f, col)] / df[
                f + '_count']  # 比例偏好
        if 'cross_{}_{}_count_ratio'.format(f, col) not in df.columns.values:
            df['cross_{}_{}_count_ratio'.format(f, col)] = df['cross_{}_{}_count'.format(f, col)] / df[
                col + '_count']  # 比例偏好
        df['cross_{}_{}_nunique_ratio_{}_count'.format(f, col, f)] = df['cross_{}_{}_nunique'.format(f, col)] / df[
            f + '_count']
        print('runtime:', time.time() - t)
    df = reduce_mem(df)
del df['id']
gc.collect()

*************************** cross feat (second order) ***************************
------------------ deviceid newsid ------------------
runtime: 149.09444093704224
------------------ deviceid pos ------------------
runtime: 159.9373984336853
------------------ deviceid netmodel ------------------
runtime: 171.00094175338745
------------------ deviceid lng_lat ------------------
runtime: 182.43015956878662
781.91 Mb, 645.73 Mb (17.42 %)
------------------ newsid deviceid ------------------
runtime: 575.3607921600342
------------------ newsid pos ------------------
runtime: 983.6665697097778
------------------ newsid netmodel ------------------
runtime: 1379.0395348072052
------------------ newsid lng_lat ------------------
runtime: 1771.5170404911041
801.37 Mb, 682.78 Mb (14.80 %)
------------------ pos deviceid ------------------
runtime: 1775.6361346244812
------------------ pos newsid ------------------
runtime: 1778.2754430770874
------------------ pos netmodel ------------------
ru

0

In [15]:
print('*************************** embedding ***************************')


# 之前有个朋友给embedding做了一个我认为非常形象的比喻：
# 在非诚勿扰上面，如果你想了解一个女嘉宾，那么你可以看看她都中意过哪些男嘉宾；
# 反过来也一样，如果你想认识一个男嘉宾，那么你也可以看看他都选过哪些女嘉宾。


def emb(df, f1, f2):
    emb_size = 8
    print('====================================== {} {} ======================================'.format(f1, f2))
    tmp = df.groupby(f1, as_index=False)[f2].agg({'{}_{}_list'.format(f1, f2): list})
    sentences = tmp['{}_{}_list'.format(f1, f2)].values.tolist()
    del tmp['{}_{}_list'.format(f1, f2)]
    for i in range(len(sentences)):
        sentences[i] = [str(x) for x in sentences[i]]
    model = Word2Vec(sentences, size=emb_size, window=5, min_count=5, sg=0, hs=1, seed=2019)
    emb_matrix = []
    for seq in sentences:
        vec = []
        for w in seq:
            if w in model:
                vec.append(model[w])
        if len(vec) > 0:
            emb_matrix.append(np.mean(vec, axis=0))
        else:
            emb_matrix.append([0] * emb_size)

    # 为了支持数组多维处理，需要先做一个变换
    emb_matrix = np.array(emb_matrix)

    for i in range(emb_size):
        tmp['{}_{}_emb_{}'.format(f1, f2, i)] = emb_matrix[:, i]
    del model, emb_matrix, sentences
    tmp = reduce_mem(tmp)
    print('runtime:', time.time() - t)
    return tmp


emb_cols = [
    ['deviceid', 'newsid'],
    ['deviceid', 'lng_lat'],
    ['newsid', 'lng_lat'],
    # ...
]
for f1, f2 in tqdm.tqdm(emb_cols):
    df = df.merge(emb(sort_df, f1, f2), on=f1, how='left')
    df = df.merge(emb(sort_df, f2, f1), on=f2, how='left')
del sort_df
gc.collect()

  0%|          | 0/3 [00:00<?, ?it/s]

*************************** embedding ***************************
====================================== deviceid newsid ======================================


INFO:MainThread:gensim.models.word2vec:collecting all words and their counts
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:MainThread:gensim.models.word2vec:collected 300717 word types from a corpus of 971439 raw words and 7049 sentences
INFO:MainThread:gensim.models.word2vec:Loading a fresh vocabulary
INFO:MainThread:gensim.models.word2vec:effective_min_count=5 retains 38716 unique words (12% of original 300717, drops 262001)
INFO:MainThread:gensim.models.word2vec:effective_min_count=5 leaves 569569 word corpus (58% of original 971439, drops 401870)
INFO:MainThread:gensim.models.word2vec:deleting the raw counts dictionary of 300717 items
INFO:MainThread:gensim.models.word2vec:sample=0.001 downsamples 0 most-common words
INFO:MainThread:gensim.models.word2vec:downsampling leaves estimated 569569 word corpus (100.0% of prior 569569)
INFO:MainThread:gensim.models.word2vec:constructing a huffman tree from 38716 words
INFO:Mai

0.54 Mb, 0.17 Mb (67.50 %)
runtime: 2007.1937282085419
====================================== newsid deviceid ======================================


INFO:MainThread:gensim.models.word2vec:collecting all words and their counts
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 210921 words, keeping 6641 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 299653 words, keeping 6846 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #30000, processed 367566 words, keeping 6925 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #40000, processed 422373 words, keeping 6965 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #50000, processed 469363 words, keeping 6980 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #60000, processed 509557 words, keeping 6990 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #70000, processed 544166 words, keeping 7002 word types

...

22.94 Mb, 8.03 Mb (65.00 %)
runtime: 2184.097326517105
====================================== lng_lat newsid ======================================


INFO:MainThread:gensim.models.word2vec:collecting all words and their counts
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #10000, processed 709405 words, keeping 249870 word types
INFO:MainThread:gensim.models.word2vec:PROGRESS: at sentence #20000, processed 790029 words, keeping 262967 word types
INFO:MainThread:gensim.models.word2vec:collected 300717 word types from a corpus of 971439 raw words and 28886 sentences
INFO:MainThread:gensim.models.word2vec:Loading a fresh vocabulary
INFO:MainThread:gensim.models.word2vec:effective_min_count=5 retains 38716 unique words (12% of original 300717, drops 262001)
INFO:MainThread:gensim.models.word2vec:effective_min_count=5 leaves 569569 word corpus (58% of original 971439, drops 401870)
INFO:MainThread:gensim.models.word2vec:deleting the raw counts dictionary of 300717 items
INFO:MainThread:gensim.models.word2vec:sample=0.00

2.20 Mb, 0.72 Mb (67.50 %)
runtime: 2213.092384815216


100%|██████████| 3/3 [03:52<00:00, 77.45s/it]


0

In [9]:
print('======================================== prepare train & valid  =============================================')
train_df = df[:train_num].reset_index(drop=True)
test_df = df[train_num:].reset_index(drop=True)
del df
gc.collect()

train_idx = train_df[train_df['day'] < 10].index.tolist()
val_idx = train_df[train_df['day'] == 10].index.tolist()

train_x = train_df.iloc[train_idx].reset_index(drop=True)
train_y = labels[train_idx]
val_x = train_df.iloc[val_idx].reset_index(drop=True)
val_y = labels[val_idx]

del train_x['day'], val_x['day'], train_df['day'], test_df['day']
gc.collect()
print('runtime:', time.time() - t)
print('========================================================================================================')



======================================== prepare train & valid  =============================================
runtime: 63.83975410461426


In [17]:
print('=============================================== training validate ===============================================')
fea_imp_list = []
clf = LGBMClassifier(
    n_jobs=10,
    learning_rate=0.01,
    n_estimators=5000,
    num_leaves=255,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=2019,
    metric=None
)

print('************** training **************')
clf.fit(
    train_x, train_y,
    eval_set=[(val_x, val_y)],
    eval_metric='auc',
    categorical_feature=cate_cols,
    early_stopping_rounds=200,
    verbose=50
)
print('runtime:', time.time() - t)

=============================================== training validate ===============================================
************** training **************


/home/ryan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['app_version', 'device_vendor', 'device_version', 'deviceid', 'lat', 'lng', 'lng_lat', 'netmodel', 'newsid', 'osversion', 'pos']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 200 rounds
[50]	valid_0's auc: 0.965979
[100]	valid_0's auc: 0.968006
[150]	valid_0's auc: 0.969476
[200]	valid_0's auc: 0.970556
[250]	valid_0's auc: 0.97144
[300]	valid_0's auc: 0.972456
[350]	valid_0's auc: 0.973145
[400]	valid_0's auc: 0.973808
[450]	valid_0's auc: 0.974351
[500]	valid_0's auc: 0.974893
[550]	valid_0's auc: 0.975298
[600]	valid_0's auc: 0.975613
[650]	valid_0's auc: 0.975821
[700]	valid_0's auc: 0.975969
[750]	valid_0's auc: 0.976131
[800]	valid_0's auc: 0.976241
[850]	valid_0's auc: 0.976334
[900]	valid_0's auc: 0.976423
[950]	valid_0's auc: 0.976494
[1000]	valid_0's auc: 0.976541
[1050]	valid_0's auc: 0.976594
[1100]	valid_0's auc: 0.976627
[1150]	valid_0's auc: 0.97666
[1200]	valid_0's auc: 0.976682
[1250]	valid_0's auc: 0.976712
[1300]	valid_0's auc: 0.976746
[1350]	valid_0's auc: 0.97676
[1400]	valid_0's auc: 0.976791
[1450]	valid_0's auc: 0.976786
[1500]	valid_0's auc: 0.976798
[1550]	valid_0's auc: 0.976803


In [10]:
### 再重新喂入新数据之前，试试save mode，看看提交的效果
from sklearn.externals import joblib
# joblib.dump(clf, path_model+'lgb.pkl')
# load model
clf = joblib.load(path_model + 'lgb.pkl')


/home/ryan/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print('************** training **************')
clf.fit(
    train_df, labels,
    eval_set=[(train_df, labels)],
    categorical_feature=cate_cols,
    verbose=50
)
print('runtime:', time.time() - t)

print('************** test predict **************')
sub = pd.read_csv(path_data + 'sample.csv')
sub['target'] = clf.predict_proba(test_df)[:, 1]
clf.predict_proba(test_df)[:, 1]
fea_imp_list.append(clf.feature_importances_)
print('runtime:', time.time() - t)

************** training **************
[50]	training's binary_logloss: 0.21329
[100]	training's binary_logloss: 0.169338
[150]	training's binary_logloss: 0.146647
[200]	training's binary_logloss: 0.133552
[250]	training's binary_logloss: 0.125048
[300]	training's binary_logloss: 0.118899
[350]	training's binary_logloss: 0.114388
[400]	training's binary_logloss: 0.110993
[450]	training's binary_logloss: 0.108129
[500]	training's binary_logloss: 0.105638
[550]	training's binary_logloss: 0.103447
[600]	training's binary_logloss: 0.101452
[650]	training's binary_logloss: 0.0996984
[700]	training's binary_logloss: 0.0981395
[750]	training's binary_logloss: 0.0967756
[800]	training's binary_logloss: 0.0955186
[850]	training's binary_logloss: 0.0943431
[900]	training's binary_logloss: 0.0932685
[950]	training's binary_logloss: 0.0922471
[1000]	training's binary_logloss: 0.0912944
[1050]	training's binary_logloss: 0.0903772
[1100]	training's binary_logloss: 0.0894798
[1150]	training's binary_l

In [25]:
gc.collect()

182

In [26]:
### 临时试试如该不对training内容，再fit一次的话，提交的效果如何
print('************** test predict **************')
sub = pd.read_csv(path_data + 'sample.csv')
# sub['target'] = clf.predict_proba(test_df)[:, 1]
clf.predict_proba(test_df)[:, 1]
fea_imp_list.append(clf.feature_importances_)
print('runtime:', time.time() - t)

************** test predict **************
runtime: 37304.49480199814


In [27]:
print('=============================================== feat importances ===============================================')
# 特征重要性可以好好看看
fea_imp_dict = dict(zip(train_df.columns.values, np.mean(fea_imp_list, axis=0)))
fea_imp_item = sorted(fea_imp_dict.items(), key=lambda x: x[1], reverse=True)
for f, imp in fea_imp_item:
    print('{} = {}'.format(f, imp))

=============================================== feat importances ===============================================
deviceid = 183689.0
newsid = 86695.0
device_version = 61750.0
lat = 31214.0
lng = 27944.0
lng_lat = 21499.0
netmodel_deviceid_lng_lat_next1_exposure_ts_gap = 6421.0
netmodel_deviceid_next1_exposure_ts_gap = 5754.0
pos = 5648.0
pos_count = 5025.0
deviceid_next3_exposure_ts_gap = 4803.0
netmodel_deviceid_lng_lat_next2_exposure_ts_gap = 3996.0
netmodel_deviceid_next3_exposure_ts_gap = 3688.0
pos_netmodel_deviceid_next1_exposure_ts_gap = 3574.0
deviceid_lng_lat_next3_exposure_ts_gap = 3547.0
pos_netmodel_deviceid_lng_lat_next1_exposure_ts_gap = 3410.0
netmodel_deviceid_next2_exposure_ts_gap = 3408.0
deviceid_next1_exposure_ts_gap = 3391.0
netmodel_deviceid_lng_lat_next3_exposure_ts_gap = 3093.0
deviceid_next2_exposure_ts_gap = 2613.0
netmodel_deviceid_next10_exposure_ts_gap = 2387.0
deviceid_count = 2259.0
deviceid_lng_lat_next1_exposure_ts_gap = 2041.0
deviceid_next5_exposure_t

In [31]:
val_y.shape

(3958109,)

In [32]:
val_x.shape

(3958109, 170)

In [37]:
val_pred.shape

(3653592,)

In [36]:
test_df.shape

(3653592, 170)

In [35]:
val_pred.shape

(3653592,)

In [28]:
print('=============================================== threshold search ===============================================')
# f1阈值敏感，所以对阈值做一个简单的迭代搜索。
t0 = 0.05
v = 0.002
best_t = t0
best_f1 = 0
for step in range(201):
    curr_t = t0 + step * v
    y = [1 if x >= curr_t else 0 for x in val_pred]
    curr_f1 = f1_score(val_y, y)
    if curr_f1 > best_f1:
        best_t = curr_t
        best_f1 = curr_f1
        print('step: {}   best threshold: {}   best f1: {}'.format(step, best_t, best_f1))
print('search finish.')

val_pred = [1 if x >= best_t else 0 for x in val_pred]
print('\nbest auc:', best_auc)
print('best f1:', f1_score(val_y, val_pred))
print('validate mean:', np.mean(val_pred))
print('runtime:', time.time() - t)


=============================================== threshold search ===============================================


ValueError: Found input variables with inconsistent numbers of samples: [3958109, 3653592]

In [ ]:
print('=============================================== sub save ===============================================')
sub.to_csv('sub_prob_{}_{}_{}.csv'.format(best_auc, best_f1, sub['target'].mean()), index=False)
sub['target'] = sub['target'].apply(lambda x: 1 if x >= best_t else 0)
sub.to_csv('sub_{}_{}_{}.csv'.format(best_auc, best_f1, sub['target'].mean()), index=False)
print('runtime:', time.time() - t)
print('finish.')
print('========================================================================================================')